In [76]:
import pandas as pd

import osmosys.osmosys
import activityinfo
from activityinfo import Client
from activityinfo import FormProcessing
import importlib
import model.modelAI
from activityinfo.id import generate_id
import json
from requests.exceptions import HTTPError
import os.path

importlib.reload(osmosys.osmosys)
importlib.reload(activityinfo.FormProcessing)
importlib.reload(model.modelAI)

<module 'model.modelAI' from 'D:\\proyectos\\OSMOSYS_2023\\IntegracionAI2023\\AI_OsmosysIntegrationScripts2023\\model\\modelAI.py'>

In [77]:
## parameters
indicatorCodeAI = 'ED2_01'
month = 'ENERO'
month_number = '2023-01'
year = 2023
test = True
print(
    '--------------------------------------' + indicatorCodeAI + '------------------------------------------------------')


--------------------------------------ED2_01------------------------------------------------------


In [78]:
# busco matchs
subformDf = osmosys.osmosys.getMatchSubforms(indicatorCodeAI)
print(subformDf.columns)
indicatorIdsOsmosys = subformDf.osmosys_indicator_id.unique()
subformSeries = subformDf.iloc[0]
formIdAI = subformSeries.form_id
parentFormIdAI = subformSeries.parent_form_id
indicatorIdAI = subformSeries.indicador_ai_id
osmosys_indicator_code = subformDf.osmosys_indicator_code.unique()
print("formIdAI: " + formIdAI)
print("parentFormIdAI: " + parentFormIdAI)
print("indicatorIdAI: " + indicatorIdAI)
print("indicatorIdsOsmosys: " + str(indicatorIdsOsmosys))
print("osmosys_indicator_code: " + str(osmosys_indicator_code))

Index(['folder', 'form', 'form_id', 'parent_form_id', 'parent_form',
       'actividad', 'indicador_ai_id', 'indicador_ai_codigo', 'indicador_ai',
       'osmosys_indicator_id', 'osmosys_indicator_code', 'osmosys_indicator',
       'osmosys_indicator_category'],
      dtype='object')
formIdAI: cdmp8ozldnik44oig0
parentFormIdAI: cpfsjn6ldnik44oig2
indicatorIdAI: cs9q1bnlcza9pfs3n
indicatorIdsOsmosys: [210 217]
osmosys_indicator_code: ['10L02' '10L09']


In [79]:
print("indicatorIdsOsmosys: " + str(indicatorIdsOsmosys))
## consulto los datos reportados, este indicador tiene 2 diferentes modos de servicio, por lo que tendra diferentes condiciones
## 210 10L02	No. de niños, niñas y adolescentes que reciben acompañamiento individual y acceden a la educación
## 217 10L09	No. de personas por las cuales ACNUR trabaja que reciben transferencias monetarias
indicatorIdsOsmosys_10L02 = [210]
indicatorIdsOsmosys_10L09 = [217]
reportedDf_10L02 = osmosys.osmosys.getIesPartnerCantonsByIndicatorsIdsOsmosysAndMonth(
    indicatorsIdsOmosys=indicatorIdsOsmosys_10L02,
    month=month, year=year)
reportedDf_10L09 = osmosys.osmosys.getIesPartnerCantonsByIndicatorsIdsOsmosysAndMonth(
    indicatorsIdsOmosys=indicatorIdsOsmosys_10L09,
    month=month, year=year)
print('reported data: ' + str(indicatorIdsOsmosys_10L02) + ': ' + str(reportedDf_10L02.shape[0]))
print('reported data: ' + str(indicatorIdsOsmosys_10L09) + ': ' + str(reportedDf_10L09.shape[0]))


indicatorIdsOsmosys: [210 217]
reported data: [210]: 1
reported data: [217]: 0


In [80]:
## obtengo los parents de AI
parentFormsIds_10L02 = FormProcessing.getParentsIds(parentFormIdAI=parentFormIdAI, reportedDf=reportedDf_10L02)
parentFormsIds_10L09 = FormProcessing.getParentsIds(parentFormIdAI=parentFormIdAI, reportedDf=reportedDf_10L09)


In [85]:
## contruyo las estructuras de datos

changesList = []
newIds = []

## 210 10L02	No. de niños, niñas y adolescentes que reciben acompañamiento individual y acceden a la educación
for index, row in parentFormsIds_10L02.iterrows():
    # print(row)
    orgAcron = row.acronym
    cantonCode = row.canton_code
    df = osmosys.osmosys.getRefValues(year=year, month=month, orgOsmosys=orgAcron,
                                      indicatorsIdsOmosys=indicatorIdsOsmosys_10L02, cantonCode=cantonCode)

    poblacion_meta = ["Refugiados/as y migrantes", "Comunidad de acogida"]

    ED2_01_RM_NA = int(df.loc[df['age_gender'] == 'NINAS'].iloc[0].value_a)
    ED2_01_RM_NN = int(df.loc[df['age_gender'] == 'NINOS'].iloc[0].value_a)
    ED2_01_RM_OTR = 0
    ED2_01_RM_ADM = 0
    ED2_01_RM_ADH = 0
    ED2_01_RM_ADOTR = 0
    ED2_01_RM_MM = int(df.loc[df['age_gender'] == 'ADULTAS'].iloc[0].value_a)
    ED2_01_RM_HH = int(df.loc[df['age_gender'] == 'ADULTOS'].iloc[0].value_a)

    dfCa = osmosys.osmosys.getCAValues(year=year, month=month, orgOsmosys=orgAcron,
                                       indicatorsIdsOmosys=indicatorIdsOsmosys_10L02, cantonCode=cantonCode)


    ED2_01_CA_NA = int(dfCa.loc[df['age_gender'] == 'NINAS'].iloc[0].value_a)
    ED2_01_CA_NN = int(dfCa.loc[df['age_gender'] == 'NINOS'].iloc[0].value_a)
    ED2_01_CA_OTR = 0
    ED2_01_CA_ADM = 0
    ED2_01_CA_ADH = 0
    ED2_01_CA_ADOTR = 0
    ED2_01_CA_MM = int(dfCa.loc[df['age_gender'] == 'ADULTAS'].iloc[0].value_a)
    ED2_01_CA_HH = int(dfCa.loc[df['age_gender'] == 'ADULTOS'].iloc[0].value_a)

    ED2_01_RM_DS = 0

    ED2_01_CA_DS = 0
    commentary = osmosys.osmosys.getCommentary(year=year, month=month, orgOsmosys=orgAcron,
                                               indicatorsIdsOmosys=indicatorIdsOsmosys_10L02).iloc[
        0].value_a

    subform = model.modelAI.SubFormED2_01(
        mes=month_number,
        colltmgkykvhxgij6=indicatorIdAI,
        rmrp='Si',
        poblacion_meta=poblacion_meta,
        modalidad_impl= 'cmo12q1kykwwosjka', ##'Provisión de servicio',
        mecanismos=None,
        transferencia=None,
        usd_transfer=None,
        ED2_01_RM_NA=ED2_01_RM_NA,
        ED2_01_RM_NN=ED2_01_RM_NN,
        ED2_01_RM_OTR=ED2_01_RM_OTR,
        ED2_01_RM_ADM=ED2_01_RM_ADM,
        ED2_01_RM_ADH=ED2_01_RM_ADH,
        ED2_01_RM_ADOTR=ED2_01_RM_ADOTR,
        ED2_01_RM_MM=ED2_01_RM_MM,
        ED2_01_RM_HH=ED2_01_RM_HH,
        ED2_01_RM_DS=ED2_01_RM_DS,
        ED2_01_CA_NA=ED2_01_CA_NA,
        ED2_01_CA_NN=ED2_01_CA_NN,
        ED2_01_CA_OTR=ED2_01_CA_OTR,
        ED2_01_CA_ADM=ED2_01_CA_ADM,
        ED2_01_CA_ADH=ED2_01_CA_ADH,
        ED2_01_CA_ADOTR=ED2_01_CA_ADOTR,
        ED2_01_CA_MM=ED2_01_CA_MM,
        ED2_01_CA_HH=ED2_01_CA_HH,
        ED2_01_CA_DS=ED2_01_CA_DS,
        ED2_01_CUAL=commentary,

    )
    newId = generate_id()
    newIds.append(newId)
    record = model.modelAI.Record(formId=formIdAI, recordId=newId, parentRecordId=row['@id'], fields=subform)
    changesList.append(record)
## 217	10L09	No. de personas por las cuales ACNUR trabaja que reciben transferencias monetarias	Educación
for index, row in parentFormsIds_10L09.iterrows():
    # print(row)
    orgAcron = row.acronym
    cantonCode = row.canton_code
    df = osmosys.osmosys.getRefValues(year=year, month=month, orgOsmosys=orgAcron,
                                      indicatorsIdsOmosys=indicatorIdsOsmosys_10L02, cantonCode=cantonCode)

    poblacion_meta = ["Refugiados/as y migrantes", "Comunidad de acogida"]

    ED2_01_RM_NA = int(df.loc[df['age_gender'] == 'NINAS'].iloc[0].value_a)
    ED2_01_RM_NN = int(df.loc[df['age_gender'] == 'NINOS'].iloc[0].value_a)
    ED2_01_RM_OTR = 0
    ED2_01_RM_ADM = 0
    ED2_01_RM_ADH = 0
    ED2_01_RM_ADOTR = 0
    ED2_01_RM_MM = int(df.loc[df['age_gender'] == 'ADULTAS'].iloc[0].value_a)
    ED2_01_RM_HH = int(df.loc[df['age_gender'] == 'ADULTOS'].iloc[0].value_a)

    dfCa = osmosys.osmosys.getCAValues(year=year, month=month, orgOsmosys=orgAcron,
                                       indicatorsIdsOmosys=indicatorIdsOsmosys_10L02, cantonCode=cantonCode)


    ED2_01_CA_NA = int(dfCa.loc[df['age_gender'] == 'NINAS'].iloc[0].value_a)
    ED2_01_CA_NN = int(dfCa.loc[df['age_gender'] == 'NINOS'].iloc[0].value_a)
    ED2_01_CA_OTR = 0
    ED2_01_CA_ADM = 0
    ED2_01_CA_ADH = 0
    ED2_01_CA_ADOTR = 0
    ED2_01_CA_MM = int(dfCa.loc[df['age_gender'] == 'ADULTAS'].iloc[0].value_a)
    ED2_01_CA_HH = int(dfCa.loc[df['age_gender'] == 'ADULTOS'].iloc[0].value_a)

    ED2_01_RM_DS = 0

    ED2_01_CA_DS = 0
    commentary = osmosys.osmosys.getCommentary(year=year, month=month, orgOsmosys=orgAcron,
                                               indicatorsIdsOmosys=indicatorIdsOsmosys_10L02).iloc[
        0].value_a
    usd_transfer = int(osmosys.osmosys.getCBIBudget(year=year, month=month, orgOsmosys=orgAcron,
                                                    indicatorsIdsOmosys=indicatorIdsOsmosys,
                                                    cantonCode=cantonCode).iloc[0].budget)
    subform = model.modelAI.SubFormED2_01(
        mes=month_number,
        colltmgkykvhxgij6=indicatorIdAI,
        rmrp='Si',
        poblacion_meta=poblacion_meta,
        modalidad_impl='c3jf6gykykwwemuk7', ## 'Efectivo',
        mecanismos= 'c3de3pmkym0dbino', ##'Entrega en ATM/Cajero automático',
        transferencia='Incondicional',
        usd_transfer=usd_transfer,
        ED2_01_RM_NA=ED2_01_RM_NA,
        ED2_01_RM_NN=ED2_01_RM_NN,
        ED2_01_RM_OTR=ED2_01_RM_OTR,
        ED2_01_RM_ADM=ED2_01_RM_ADM,
        ED2_01_RM_ADH=ED2_01_RM_ADH,
        ED2_01_RM_ADOTR=ED2_01_RM_ADOTR,
        ED2_01_RM_MM=ED2_01_RM_MM,
        ED2_01_RM_HH=ED2_01_RM_HH,
        ED2_01_RM_DS=ED2_01_RM_DS,
        ED2_01_CA_NA=ED2_01_CA_NA,
        ED2_01_CA_NN=ED2_01_CA_NN,
        ED2_01_CA_OTR=ED2_01_CA_OTR,
        ED2_01_CA_ADM=ED2_01_CA_ADM,
        ED2_01_CA_ADH=ED2_01_CA_ADH,
        ED2_01_CA_ADOTR=ED2_01_CA_ADOTR,
        ED2_01_CA_MM=ED2_01_CA_MM,
        ED2_01_CA_HH=ED2_01_CA_HH,
        ED2_01_CA_DS=ED2_01_CA_DS,
        ED2_01_CUAL=commentary,

    )
    newId = generate_id()
    newIds.append(newId)
    record = model.modelAI.Record(formId=formIdAI, recordId=newId, parentRecordId=row['@id'], fields=subform)
    changesList.append(record)

print('changes to charge: ' + str(len(changesList)))

changes to charge: 1


In [86]:
## creo respaldo
changes = model.modelAI.Changes(changesList)
finalJson = json.dumps(changes, default=model.modelAI.default)
print(finalJson)

{"changes": [{"formId": "cdmp8ozldnik44oig0", "recordId": "cle4minym0003ocmni4t03epk", "parentRecordId": "cle2zs2dt00fpw0mnrz6st0k9", "fields": {"mes": "2023-01", "colltmgkykvhxgij6": "cs9q1bnlcza9pfs3n", "rmrp": "Si", "poblacion_meta": ["Refugiados/as y migrantes", "Comunidad de acogida"], "modalidad_impl": "Provisi\u00f3n de servicio", "mecanismos": null, "transferencia": null, "usd_transfer": null, "ED2_01_RM_NA": 23, "ED2_01_RM_NN": 10, "ED2_01_RM_OTR": 0, "ED2_01_RM_ADM": 0, "ED2_01_RM_ADH": 0, "ED2_01_RM_ADOTR": 0, "ED2_01_RM_MM": 0, "ED2_01_RM_HH": 0, "ED2_01_RM_DS": 0, "ED2_01_CA_NA": 0, "ED2_01_CA_NN": 0, "ED2_01_CA_OTR": 0, "ED2_01_CA_ADM": 0, "ED2_01_CA_ADH": 0, "ED2_01_CA_ADOTR": 0, "ED2_01_CA_MM": 0, "ED2_01_CA_HH": 0, "ED2_01_CA_DS": 0, "ED2_01_CUAL": "EN LAS OFICINAS DE HIAS SE REALIZA EL PROCESO DE INSERCI\u00d3N ESCOLAR A TRAV\u00c9S DE LA MATRICULA EXTRAORDINARIA Y DEL PROCESO DE APRESTAMIENTO"}}]}


In [83]:
#open text file
try:
    os.mkdir(month)
except Exception:
    pass
text_file = open(os.path.join(month, "data_" + indicatorCodeAI + '-' + str(indicatorIdsOsmosys) + ".json"), "w")
#write string to file
n = text_file.write(finalJson)

#close file
text_file.close()
print(" creado respaldo: " + text_file.name)
newIdsDict = {"newIds": newIds}
newIdsDf = pd.DataFrame(newIdsDict)
newIdsDf.to_csv(os.path.join(month, "new_ids_" + indicatorCodeAI + ".csv"))

 creado respaldo: ENERO\data_ED2_01-[210 217].json


In [84]:
## envio a AI
if (False):
    print(
        '--------------------------------------' + indicatorCodeAI + '-' + str(
            indicatorIdsOsmosys) + '------------------------------------------------------')
else:
    try:
        if len(changesList) > 0:
            print('se envia a AI')
            client = Client(token=osmosys.osmosys.getToken(), base_url='https://www.activityinfo.org/resources')
            client.post_resource(path='update', body=finalJson)
            print(
                '--------------------------------------' + indicatorCodeAI + '-' + str(
                    indicatorIdsOsmosys) + '------------------------------------------------------')
        else:
            print('nada que enviar a AI')
    except HTTPError as e:
        code = e.response.status_code
        print('error:')
        print(code)

        print(e)

se envia a AI
--------------------------------------ED2_01-[210 217]------------------------------------------------------
